In [ ]:
# =============================================
# [3/9] LSTM & GRU
# =============================================
# 목표: 장기 의존성 문제를 해결한 LSTM, GRU의 성능을 RNN과 비교합니다.

# --- 1. 기본 설정 (이전 단계와 유사) ---
!pip install torch torchtext transformers datasets scikit-learn matplotlib

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import time

# --- 2. 데이터 준비 (이전 단계와 동일) ---
# (02_rnn_classification.ipynb의 데이터 준비 코드를 여기에 그대로 붙여넣으세요.)
# ... (생략) ...
# 데이터 로더까지 준비되었다고 가정합니다.

# --- 3. LSTM / GRU 모델 정의 ---
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # nn.LSTM 사용
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids)
        # LSTM은 hidden state와 cell state를 튜플로 반환
        # output: (batch, seq_len, hidden_dim), (hidden, cell): ((n_layers, batch, hidden), (n_layers, batch, hidden))
        output, (hidden, cell) = self.lstm(embedded)
        # 마지막 레이어의 hidden state 사용
        last_hidden = hidden[-1, :, :]
        return self.fc(last_hidden)

class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, n_layers=1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # nn.GRU 사용
        self.gru = nn.GRU(embed_dim, hidden_dim, num_layers=n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids)
        # GRU는 hidden state만 반환
        output, hidden = self.gru(embedded)
        last_hidden = hidden[-1, :, :]
        return self.fc(last_hidden)

# --- 4. 모델 학습 및 평가 함수 ---
def train_and_evaluate(model, train_loader, test_loader, epochs=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    start_time = time.time()

    for epoch in range(epochs):
        model.train()
        for batch in train_loader:
            input_ids, attention_mask, labels = batch
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)
            total_acc += (preds == labels).sum().item()
            total_count += labels.size(0)

    end_time = time.time()
    accuracy = total_acc / total_count
    print(f"Test Accuracy: {accuracy:.4f}, Training time: {end_time - start_time:.2f}s")
    return accuracy

# --- 5. 성능 비교 실험 ---
# 하이퍼파라미터 (은닉 크기, 레이어 수 튜닝 가능)
VOCAB_SIZE = tokenizer.vocab_size
EMBED_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 2
N_LAYERS = 2 # 레이어 수를 2로 늘려 실험

# RNN 모델 (비교를 위해 다시 정의)
# ... (02_rnn_classification.ipynb의 RNNClassifier 클래스 코드) ...
# rnn_model = RNNClassifier(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM)
# print("--- RNN Performance ---")
# train_and_evaluate(rnn_model, train_loader, test_loader)

# LSTM 모델
lstm_model = LSTMClassifier(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS)
print("\n--- LSTM Performance ---")
train_and_evaluate(lstm_model, train_loader, test_loader)

# GRU 모델
gru_model = GRUClassifier(VOCAB_SIZE, EMBED_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS)
print("\n--- GRU Performance ---")
train_and_evaluate(gru_model, train_loader, test_loader)

print("\n결론: 일반적으로 LSTM과 GRU가 RNN보다 더 높은 성능을 보이며, 장기 의존성 포착에 유리합니다.")